# 🧠 Iterative Prompt Development

In this notebook, we’ll explore how to iteratively refine prompts to improve the quality of LLM outputs, focusing on **business-relevant content generation**.

We'll use a realistic case study: generating marketing and technical copy for an AI-powered **risk assessment tool** used by financial and compliance teams.

This mirrors real-world workflows where data scientists:
- Collaborate with product/marketing to document models
- Tune prompts for reports, dashboards, and client-facing content
- Structure outputs for usability in apps or portals

---

### 🛠️ Stack:
- OpenAI GPT-4 via `openai` Python client
- Python utilities for formatting and inspection
- HTML rendering for structured responses

---

We'll take a raw **fact sheet** and refine our prompt step by step — just like tuning a machine learning model or pipeline.


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Retrieve the API key
api_key = os.getenv("OPENAI_API_KEY")

# Sanity check (should print a masked version)
if api_key:
    print("✅ API key loaded successfully.")
else:
    print("❌ API key not found. Please check your .env file.")

In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key=api_key)


In [ ]:
def get_completion(prompt, model="gpt-4o", temperature=0):
    """
    Helper function to call OpenAI Chat Completion API with a given prompt.
    Defaults to deterministic output (temperature=0).
    """
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    return response.choices[0].message.content


In [ ]:
fact_sheet_product = """
PRODUCT NAME
- RiskIntelliScore™

OVERVIEW
- A predictive analytics tool for early risk detection in corporate client portfolios.
- Designed to flag financial distress, compliance violations, or operational anomalies.
- Integrates seamlessly with existing enterprise data pipelines and reporting dashboards.
- Ideal for compliance teams, credit analysts, and enterprise risk officers.

FEATURES
- Multi-source data ingestion: structured + unstructured
- Real-time scoring engine (based on gradient-boosted decision trees)
- Customizable thresholds per industry segment
- Supports JSON and Parquet formats

ALGORITHMIC CORE
- Ensemble model combining:
    - Financial ratio analysis (Altman Z-score derived features)
    - NLP analysis on legal filings and earnings calls
    - Time series anomaly detection for operational KPIs

TECHNICAL DETAILS
- Built on Python, scikit-learn, and FastAPI
- Average model latency: 180 ms per request (batch mode available)
- Integrates with Azure, GCP, and AWS
- Model versioning and drift detection built-in

COMPLIANCE & AUDIT
- ISO 27001 and SOC 2 Type II certified
- Full audit trail of predictions and data sources
- Supports GDPR and CCPA compliance

PRODUCT CODES
- RIS-PRM-001 (Premium Tier)
- RIS-STD-001 (Standard Tier)

COUNTRY OF ORIGIN
- Netherlands
"""


## 📝 Prompt 1: Generate a General Product Description

Let’s start with a basic instruction to generate a product description from the technical fact sheet.

We'll use the raw product sheet for *RiskIntelliScore™*, and ask the model to summarize it into a short, marketing-friendly description — without placing any specific constraints.

This simulates an initial attempt to draft a client-facing summary based on raw internal documentation.


In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
description for a financial services product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: 
```
{fact_sheet_product}
```
"""

print(prompt)

In [ ]:
response = get_completion(prompt)
print(response)


## ✂️ Prompt 2: Limit Description to 50 Words

In many enterprise settings, product summaries must fit into:
- Compact UI widgets
- Executive dashboards
- Email preview text
- SEO-snippets

We'll refine the prompt to cap the length of the output at **50 words** while still capturing the essence of the product.

This iteration helps align model output with UI/UX constraints or content guidelines.


In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
concise product description for a financial services tool.

Write a product description based on the technical fact sheet 
delimited by triple backticks.

Use at most 50 words.

Technical specifications: 
```
{fact_sheet_product}
```
"""

response = get_completion(prompt)
print(response)
print("\n🧮 Word count:", len(response.split()))


## 🧑‍💻 Prompt 3: Focus on Technical Features

Sometimes, the goal isn’t flashy marketing — it's to inform a technically sophisticated audience who care about:
- Algorithms used
- Integrations
- Compliance and latency guarantees

We'll now adjust the prompt to shift focus from general marketing to technical depth, while keeping the 50-word constraint in place.

This aligns with use cases like:
- Internal documentation
- Sales engineering material
- API marketplace descriptions


In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
product description for a financial risk analytics tool.

Write a product description based on the technical fact sheet 
delimited by triple backticks.

The description is intended for technical users (e.g. ML engineers, data scientists, or enterprise architects), 
so it should emphasize the algorithmic design, data integration, and deployment flexibility.

Use at most 50 words.

Technical specifications: 
```
{fact_sheet_product}
```
"""

response = get_completion(prompt)
print(response)
print("\n🧮 Word count:", len(response.split()))


## 🏷️ Prompt 4: Include Product Codes in Output

Many enterprise tools have multiple SKUs or product tiers. Including these codes in descriptions helps:

- Link marketing copy to ordering systems
- Auto-generate tags for pricing, APIs, or billing
- Support backend integrations (catalogs, CRM, etc.)

In this prompt, we ask the model to:
- Focus on technical aspects (as before)
- Include any **7-character Product IDs** from the fact sheet


In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
product description for a financial risk analytics tool.

Write a product description based on the technical fact sheet 
delimited by triple backticks.

The description is intended for technical users (e.g. ML engineers, data scientists, or enterprise architects), 
so it should emphasize the algorithmic design, data integration, and deployment flexibility.

At the end of the description, include any 7-character Product ID(s) listed in the fact sheet.

Use at most 50 words.

Technical specifications: 
```
{fact_sheet_product}
```
"""

response = get_completion(prompt)
print(response)


## 🧱 Prompt 5: Add Technical Metrics Table (in HTML)

In this step, we’ll ask the model to return:
- A short technical product description
- A structured HTML `<table>` listing performance, integrations, and compliance highlights

Why?
- Helps in auto-generating product pages
- Easily embeds into CMS or internal tools
- Useful for developers consuming docs or catalogs

We'll also wrap the text in a `<div>` element for layout control.


In [ ]:
prompt = f"""
Your task is to help a marketing or product documentation team create an HTML summary 
for a financial risk analytics product.

Write a short product description based on the technical fact sheet 
delimited by triple backticks.

Focus on the model architecture, latency, data integration formats, 
and compliance certifications.

After the description, include a table with two columns:
- Column 1: Feature
- Column 2: Detail

Only include technical features like model type, average latency, supported formats, cloud compatibility, compliance, etc.

Wrap the description in a <div> tag and the table in <table> tags. Enclose the code in ```html codeblock.

Technical specifications: ```{fact_sheet_product}```
"""

html_response = get_completion(prompt)
print(html_response)

with open('page.html', 'w') as fw:
    fw.write(html_response.replace("```html", '').replace("```", ''))

In [ ]:
from IPython.display import display, HTML
import re

# Extract HTML code from triple backticks
match = re.search(r"```html\s*(.*?)```", html_response, re.DOTALL)
if match:
    clean_html = match.group(1)
    display(HTML(clean_html))
else:
    print("No valid HTML code block found in response.")
